<img src="Figures/Deephyper.png" style="height: 200px;">

<h1><center>Hyperparameter search for classification with Tabular data</center></h1>

<div class="alert alert-info">
    
<b>Reference</b>
    
This tutorial is based on materials from the Keras Documentation:
* [Structured data classification from scratch](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)
    
</div>


<div class="alert alert-warning">

<b>Warning</b>
    
By design asyncio does not allow nested event loops. Jupyter is using Tornado which already starts an event loop. Therefore the following patch is required to run this tutorial.
    
This tutorial should be run with `tensorFlow>=2.6`.
    
</div>

In [1]:
!pip install nest_asyncio

import nest_asyncio
nest_asyncio.apply()

<div class="alert alert-info">
    
<b>Note</b>
    
The following environment variables can be used to avoid the logging of **some** Tensorflow *DEBUG*, *INFO* and *WARNING* statements.
    
</div>

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = str(3)
os.environ["AUTOGRAPH_VERBOSITY"] = str(0)

## Imports

<div class="alert alert-block alert-danger">
    
<b>Danger</b> 

The following cell contains Tensorflow import `import tensorflow as tf`. It is important to follow this strategy instead of `from tensorflow.keras.layers import ...` to avoid non-serializable data, creating crashed during the search. For example, the original Keras tutorial was using the following set of imports which was creating a serialization error in our use case.
    
```python
from tensorflow import keras
from tensorflow.keras import layers
...
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
```
    
</div>

In [3]:
import ray
import pandas as pd
import tensorflow as tf

<div class="alert alert-info">
    
<b>Note</b>
    
The following can be used to detect if **GPU** devices are available on the current host. Therefore, this notebook will automatically adapt the parallel execution based on the ressources available locally. However, it will not be the case if many compute nodes are requested.
    
</div>

In [4]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == "GPU"]

n_gpus = len(get_available_gpus())
is_gpu_available = n_gpus > 0

if is_gpu_available:
    print(f"{n_gpus} GPU{'s are' if n_gpus > 1 else ' is'} available.")
else:
    print("No GPU available")

No GPU available


### The dataset (from Keras.io)

The [dataset](https://archive.ics.uci.edu/ml/datasets/heart+Disease) is provided by the
Cleveland Clinic Foundation for Heart Disease.
It's a CSV file with 303 rows. Each row contains information about a patient (a
**sample**), and each column describes an attribute of the patient (a **feature**). We
use the features to predict whether a patient has a heart disease (**binary
classification**).

Here's the description of each feature:

Column| Description| Feature Type
------------|--------------------|----------------------
Age | Age in years | Numerical
Sex | (1 = male; 0 = female) | Categorical
CP | Chest pain type (0, 1, 2, 3, 4) | Categorical
Trestbpd | Resting blood pressure (in mm Hg on admission) | Numerical
Chol | Serum cholesterol in mg/dl | Numerical
FBS | fasting blood sugar in 120 mg/dl (1 = true; 0 = false) | Categorical
RestECG | Resting electrocardiogram results (0, 1, 2) | Categorical
Thalach | Maximum heart rate achieved | Numerical
Exang | Exercise induced angina (1 = yes; 0 = no) | Categorical
Oldpeak | ST depression induced by exercise relative to rest | Numerical
Slope | Slope of the peak exercise ST segment | Numerical
CA | Number of major vessels (0-3) colored by fluoroscopy | Both numerical & categorical
Thal | 3 = normal; 6 = fixed defect; 7 = reversible defect | Categorical
Target | Diagnosis of heart disease (1 = true; 0 = false) | Target

In [5]:
def load_data():
#     file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
    file_url = "heart.csv"
    dataframe = pd.read_csv(file_url)

    val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
    train_dataframe = dataframe.drop(val_dataframe.index)

    return train_dataframe, val_dataframe


def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

## Preprocessing & encoding of features

The next cells use `tf.keras.layers.Normalization()` to apply standard scaling on the features. Then, the `tf.keras.layers.StringLookup` and `tf.keras.layers.IntegerLookup` are used to encode categorical variables.

In [6]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = (
        tf.keras.layers.StringLookup if is_string else tf.keras.layers.IntegerLookup
    )
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

## Define the run-function

The run-function defines how the objective that we want to maximize is computed. It takes a `config` dictionnary as input and often returns a scalar value that we want to maximize. The `config` contains a sample value of hyperparameters to we want to tune. In this example we will search for:
* `units` (default value: `32`)
* `activation` (default value: `"relu"`)
* `dropout_rate` (default value: `0.5`)
* `num_epochs` (default value: `50`)
* `batch_size` (default value: `32`)
* `learning_rate` (default value: `1e-3`)
An hyperparameter value can be acessed easily in the dictionnary through the corresponding key, for example `config["units"]`.

In [7]:
def run(config: dict):
    tf.autograph.set_verbosity(0)
    
    train_dataframe, val_dataframe = load_data()

    train_ds = dataframe_to_dataset(train_dataframe)
    val_ds = dataframe_to_dataset(val_dataframe)

    train_ds = train_ds.batch(config["batch_size"])
    val_ds = val_ds.batch(config["batch_size"])

    # Categorical features encoded as integers
    sex = tf.keras.Input(shape=(1,), name="sex", dtype="int64")
    cp = tf.keras.Input(shape=(1,), name="cp", dtype="int64")
    fbs = tf.keras.Input(shape=(1,), name="fbs", dtype="int64")
    restecg = tf.keras.Input(shape=(1,), name="restecg", dtype="int64")
    exang = tf.keras.Input(shape=(1,), name="exang", dtype="int64")
    ca = tf.keras.Input(shape=(1,), name="ca", dtype="int64")

    # Categorical feature encoded as string
    thal = tf.keras.Input(shape=(1,), name="thal", dtype="string")

    # Numerical features
    age = tf.keras.Input(shape=(1,), name="age")
    trestbps = tf.keras.Input(shape=(1,), name="trestbps")
    chol = tf.keras.Input(shape=(1,), name="chol")
    thalach = tf.keras.Input(shape=(1,), name="thalach")
    oldpeak = tf.keras.Input(shape=(1,), name="oldpeak")
    slope = tf.keras.Input(shape=(1,), name="slope")

    all_inputs = [
        sex,
        cp,
        fbs,
        restecg,
        exang,
        ca,
        thal,
        age,
        trestbps,
        chol,
        thalach,
        oldpeak,
        slope,
    ]

    # Integer categorical features
    sex_encoded = encode_categorical_feature(sex, "sex", train_ds, False)
    cp_encoded = encode_categorical_feature(cp, "cp", train_ds, False)
    fbs_encoded = encode_categorical_feature(fbs, "fbs", train_ds, False)
    restecg_encoded = encode_categorical_feature(restecg, "restecg", train_ds, False)
    exang_encoded = encode_categorical_feature(exang, "exang", train_ds, False)
    ca_encoded = encode_categorical_feature(ca, "ca", train_ds, False)

    # String categorical features
    thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)

    # Numerical features
    age_encoded = encode_numerical_feature(age, "age", train_ds)
    trestbps_encoded = encode_numerical_feature(trestbps, "trestbps", train_ds)
    chol_encoded = encode_numerical_feature(chol, "chol", train_ds)
    thalach_encoded = encode_numerical_feature(thalach, "thalach", train_ds)
    oldpeak_encoded = encode_numerical_feature(oldpeak, "oldpeak", train_ds)
    slope_encoded = encode_numerical_feature(slope, "slope", train_ds)

    all_features = tf.keras.layers.concatenate(
        [
            sex_encoded,
            cp_encoded,
            fbs_encoded,
            restecg_encoded,
            exang_encoded,
            slope_encoded,
            ca_encoded,
            thal_encoded,
            age_encoded,
            trestbps_encoded,
            chol_encoded,
            thalach_encoded,
            oldpeak_encoded,
        ]
    )
    x = tf.keras.layers.Dense(config["units"], activation=config["activation"])(
        all_features
    )
    x = tf.keras.layers.Dropout(config["dropout_rate"])(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(all_inputs, output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    model.compile(optimizer, "binary_crossentropy", metrics=["accuracy"])

    history = model.fit(
        train_ds, epochs=config["num_epochs"], validation_data=val_ds, verbose=0
    )

    return history.history["val_accuracy"][-1]

<div class="alert alert-success">
    
<b>Important</b>
    
The objective maximised by DeepHyper is the scalar value returned by the `run`-function. In this tutorial it corresponds to the validation accuracy of the last epoch of training which we retrieve in the `History` object returned by the `model.fit(...)` call.
    
```python
...
history = model.fit(
    train_ds, epochs=config["num_epochs"], validation_data=val_ds, verbose=0
)

return history.history["val_accuracy"][-1]
...
```

Using an objective like `max(history.history["val_accuracy"])` can have undesired effect such as training curves passing by a local maximum and then dropping which will not generate a model in capacity of ingesting well more data in the future.
    
</div>



## Evaluate a default configuration

We evaluate the performance of the default set of hyperparameters provided in the Keras tutorial.

In [8]:
# We define a dictionnary for the default values
default_config = {
    "units": 32,
    "activation": "relu",
    "dropout_rate": 0.5,
    "num_epochs": 50,
    "batch_size": 32,
    "learning_rate": 1e-3,
}

# We launch the Ray run-time depending of the detected local ressources
# and execute the `run` function with the default configuration
# WARNING: in the case of GPUs it is important to follow this scheme
# to avoid multiple processes (Ray workers vs current process) to lock
# the same GPU.
if is_gpu_available:
    
    if not(ray.is_initialized()):
        ray.init(num_cpus=n_gpus, num_gpus=n_gpus, log_to_driver=False)
    
    run_default = ray.remote(num_cpus=1, num_gpus=1)(run)
    
    objective_default = ray.get(run_default.remote(default_config))
    
else:
    
    if not(ray.is_initialized()):
        ray.init(num_cpus=1, log_to_driver=False)
    
    run_default = run
    
    objective_default = run_default(default_config)
    
print(f"Accuracy Default Configuration:  {objective_default:.3f}")

2021-09-27 18:00:57,707	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


Accuracy Default Configuration:  0.820


## Define the Hyperparameter optimization problem

Hyperparameter ranges are defined using the following syntax:

* Discrete integer ranges are generated from a tuple `(lower: int, upper: int)`
* Continuous prarameters are generated from a tuple `(lower: float, upper: float)`
* Categorical or nonordinal hyperparameter ranges can be given as a list of possible values `[val1, val2, ...]`

We provide the default configuration of hyperparameters as a starting point of the problem.

In [9]:
from deephyper.problem import HpProblem


problem = HpProblem()

# Discrete hyperparameter (sampled with uniform prior)
problem.add_hyperparameter((8, 128), "units")

# Categorical hyperparameter (sampled with uniform prior)
ACTIVATIONS = ["elu", "gelu", "hard_sigmoid", "linear", "relu", "selu",
    "sigmoid", "softplus", "softsign", "swish", "tanh",
]
problem.add_hyperparameter(ACTIVATIONS, "activation")

# Real hyperparameter (sampled with uniform prior)
problem.add_hyperparameter((0.0, 0.6), "dropout_rate")

problem.add_hyperparameter((10, 100), "num_epochs")

# Discrete and Real hyperparameters (sampled with log-uniform)
problem.add_hyperparameter((8, 256, "log-uniform"), "batch_size")
problem.add_hyperparameter((1e-5, 1e-2, "log-uniform"), "learning_rate")

# Add a starting point to try first
problem.add_starting_point(**default_config)

problem

Configuration space object:
  Hyperparameters:
    activation, Type: Categorical, Choices: {elu, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softplus, softsign, swish, tanh}, Default: elu
    batch_size, Type: UniformInteger, Range: [8, 256], Default: 45, on log-scale
    dropout_rate, Type: UniformFloat, Range: [0.0, 0.6], Default: 0.3
    learning_rate, Type: UniformFloat, Range: [1e-05, 0.01], Default: 0.0003162278, on log-scale
    num_epochs, Type: UniformInteger, Range: [10, 100], Default: 55
    units, Type: UniformInteger, Range: [8, 128], Default: 68


  Starting Point:
{0: {'activation': 'relu',
     'batch_size': 32,
     'dropout_rate': 0.5,
     'learning_rate': 0.001,
     'num_epochs': 50,
     'units': 32}}

## Define the evaluator object

The `Evaluator` object allows to change the parallelization backend used by DeepHyper. It is a standalone object which schedule the execution of remote tasks. All evaluators needs a `run_function` to be instantiated. Then a keyword `method` defines the backend (e.g., `"ray"`) and the `method_kwargs` corresponds to keyword arguments of this chosen `method`.

```python
evaluator = Evaluator.create(run_function, method, method_kwargs)
```

Once created the `evaluator.num_workers` gives access to the number of available parallel workers.

Finally, to submit and collect tasks to the evaluator one just needs to use the following interface:

```python
configs = [...]
evaluator.submit(configs)
...
tasks_done = evaluator.get("BATCH", size=1) # For asynchronous
tasks_done = evaluator.get("ALL") # For batch synchronous
```

<div class="alert alert-warning">

<b>Warning</b>

Each `Evaluator` saves its own state, therefore it is crutial to create a new evaluator when launching a fresh search.
    
</div>


In [10]:
from deephyper.evaluator.evaluate import Evaluator
from deephyper.evaluator.callback import LoggerCallback


def get_evaluator(run_function):
    
    # Default arguments for Ray: 1 worker and 1 worker per evaluation
    method_kwargs = {
        "num_cpus": 1, 
        "num_cpus_per_task": 1,
        "callbacks": [LoggerCallback()]
    }

    # If GPU devices are detected then it will create 'n_gpus' workers
    # and use 1 worker for each evaluation
    if is_gpu_available:
        method_kwargs["num_cpus"] = n_gpus
        method_kwargs["num_gpus"] = n_gpus
        method_kwargs["num_cpus_per_task"] = 1
        method_kwargs["num_gpus_per_task"] = 1

    evaluator = Evaluator.create(
        run_function, 
        method="ray", 
        method_kwargs=method_kwargs
    )
    print(f"Created new evaluator with {evaluator.num_workers} worker{'s' if evaluator.num_workers > 1 else ''} and config: {method_kwargs}", )
    
    return evaluator

evaluator_1 = get_evaluator(run)

Created new evaluator with 1 worker and config: {'num_cpus': 1, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.LoggerCallback object at 0x7f875b2355e0>]}


## Define and run the asynchronous model-based search

A primary pillar of hyperparameter search in DeepHyper is given by an asynchronous parallel model-based search paradigm (henceforth AMBS). AMBS may be described in the following algorithm:

<img src="Figures/AMBS.png" style="width: 500px;">

Following the parallelized evaluation of these configurations, a low-fidelity and high efficiency model (henceforth "the surrogate") is devised to reproduce the relationship between the input variables involved in the model (i.e., the choice of hyperparameters) and the outputs (which are generally a measure of validation data accuracy). After obtaining this surrogate of the validation accuracy, we may utilize ideas from classical methods in Bayesian optimization literature for adaptively sample the search space of hyperparameters.

First, the surrogate is used to obtain an estimate for the mean value of the validation accuracy at a certain sampling location $x$ in addition to an estimated variance. The latter requirement restricts us to the use of high efficiency data-driven modeling strategies that have inbuilt variance estimates (such as a Gaussian process or Random Forest regressor). Regions where the mean is high represent opportunities for exploitation and regions where the variance is high represent opportunities for exploration. An optimistic acquisition function called UCB can be constructed using these two quantities:

$$L_{\text{UCB}}(x) = \mu(x) + \kappa \cdot \sigma(x)$$

The *unevaluated* hyperparameter configurations that *maximize* the acquisition function are chosen for the next batch of evaluations. Note that the choice of the variance weighting parameter $\kappa$ controls the degree of exploration in the hyperparameter search with zero indicating purely exploitation (unseen configurations where the predicted accuracy is highest will be sampled). The top `s` configurations are selected for the new batch. The following schematic demonstrates this process:

<img src="Figures/BO_AF.png" width="500">

The process of obtaining `s` configurations relies on the "constant-liar" strategy where a sampled configuration is mapped to a dummy output given by a bulk metric of all the evaluated configurations thus far (such as the maximum, mean or median validation accuracy). Prior to sampling the next configuration by acquisition function maximization, the surrogate is retrained with the dummy output as a data point. As the true validation accuracy becomes available for one of the sampled configurations, the dummy output is replaced and the surrogate is updated. This allows for scalable asynchronous (or batch synchronous) sampling of new hyperparameter configurations. 

####  Choice of surrogate model

Users should note that our choice of the surrogate is given by the Random Forest regressor due to its ability to handle non-ordinal data (hyperparameter configurations may not be purely continuous or even numerical). Evidence for how they outperform other methods (such as Gaussian processes) is also available in [1]

<img src="Figures/RFR.png" width="400">

<img src="Figures/RFR_Superior.png" width="800">


In [11]:
from deephyper.search.hps import AMBS

# Uncomment the following line to show the arguments of AMBS.
# AMBS?

In [12]:
# Instanciate the search with the problem and a specific evaluator
search = AMBS(problem, evaluator_1)

<div class="alert alert-info">
    
<b>Note</b>
    
All DeepHyper's search algorithm have two stopping criteria:

* `max_evals`: (*int*) which defines the maximum number of evaluations that we want to perform. Default to `-1` for an infinite number.
* `timeout`: (positive *int*) which defines a time budget (in secondes) before stopping the search. Default to `None` for an infinite time budget.
    
</div>

In [13]:
results = search.search(max_evals=10)

[00001] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00002] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00003] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00004] -- best objective: 0.8196721076965332 -- received objective: 0.7868852615356445
[00005] -- best objective: 0.8196721076965332 -- received objective: 0.7868852615356445
[00006] -- best objective: 0.8524590134620667 -- received objective: 0.8524590134620667
[00007] -- best objective: 0.8524590134620667 -- received objective: 0.8360655903816223
[00008] -- best objective: 0.8524590134620667 -- received objective: 0.7704917788505554
[00009] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889
[00010] -- best objective: 0.8524590134620667 -- received objective: 0.24590164422988892


<div class="alert alert-warning">

<b>Warning</b>
    
The `search` call does not output any information about the current status of the search. However, `results.csv` file is created in the local directly and can be visualized to see finished tasks.
    
</div>

The returned `results` is a Pandas Dataframe where columns are hyperparameters and information stored by the evaluator:

* `id` is a unique identifier corresponding to the order of creation of tasks
* `objective` is the value returned by the run-function
* `elapsed_sec` is the time (in secondes) when the task completed since the creation of the evaluator.
* `duration` is the duration (in secondes) of the task to be computed.

In [14]:
results

,activation,batch_size,dropout_rate,learning_rate,num_epochs,units,id,objective,elapsed_sec,duration
0,relu,32,0.500000,0.001000,50,32,1,0.819672,6.230122,6.206514
1,hard_sigmoid,42,0.540868,0.000824,51,70,2,0.819672,10.061022,3.629809
2,hard_sigmoid,11,0.022177,0.000412,60,124,3,0.819672,15.157963,4.885490
3,tanh,9,0.423570,0.000035,44,122,4,0.786885,19.546616,4.212101
4,softplus,36,0.536837,0.000119,94,65,5,0.786885,24.468021,4.746074
5,softplus,10,0.265353,0.000790,19,21,6,0.852459,27.508826,2.867473
6,tanh,9,0.349124,0.000289,18,14,7,0.836066,30.627359,2.923309
7,relu,243,0.090046,0.000832,93,13,8,0.770492,35.201301,4.289235
8,elu,254,0.220927,0.006949,27,11,9,0.803279,39.339825,3.830671
9,relu,8,0.463332,0.000019,11,45,10,0.245902,42.353928,2.770527


The search can be continued without any issue.

In [15]:
results = search.search(max_evals=5)

results

[00011] -- best objective: 0.8524590134620667 -- received objective: 0.7704917788505554
[00012] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889
[00013] -- best objective: 0.8524590134620667 -- received objective: 0.7704917788505554
[00014] -- best objective: 0.8524590134620667 -- received objective: 0.6557376980781555
[00015] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889


,activation,batch_size,dropout_rate,learning_rate,num_epochs,units,id,objective,elapsed_sec,duration
0,relu,32,0.500000,0.001000,50,32,1,0.819672,6.230122,6.206514
1,hard_sigmoid,42,0.540868,0.000824,51,70,2,0.819672,10.061022,3.629809
2,hard_sigmoid,11,0.022177,0.000412,60,124,3,0.819672,15.157963,4.885490
3,tanh,9,0.423570,0.000035,44,122,4,0.786885,19.546616,4.212101
4,softplus,36,0.536837,0.000119,94,65,5,0.786885,24.468021,4.746074
5,softplus,10,0.265353,0.000790,19,21,6,0.852459,27.508826,2.867473
6,tanh,9,0.349124,0.000289,18,14,7,0.836066,30.627359,2.923309
7,relu,243,0.090046,0.000832,93,13,8,0.770492,35.201301,4.289235
8,elu,254,0.220927,0.006949,27,11,9,0.803279,39.339825,3.830671
9,relu,8,0.463332,0.000019,11,45,10,0.245902,42.353928,2.770527


Now that the search is over, let us print the best configuration found during this run.

In [16]:
i_max = results.objective.argmax()
best_config = results.iloc[i_max][:-3].to_dict()

print(f"The default configuration has an accuracy of {objective_default:.3f}. " 
      f"The best configuration found by DeepHyper has an accuracy {results['objective'].iloc[i_max]:.3f}, " 
      f"trained in {results['duration'].iloc[i_max]:.2f} secondes and "
      f"finished after {results['elapsed_sec'].iloc[i_max]:.2f} secondes of search.")



best_config

The default configuration has an accuracy of 0.820. The best configuration found by DeepHyper has an accuracy 0.852, trained in 2.87 secondes and finished after 27.51 secondes of search.


{'activation': 'softplus',
 'batch_size': 10,
 'dropout_rate': 0.2653530711115132,
 'learning_rate': 0.0007904971222891,
 'num_epochs': 19,
 'units': 21,
 'id': 6}

## Restart from a checkpoint

It can often be useful to continue the search from previous results. For example, if the allocation requested was not enough or if an unexpected crash happened. The `AMBS` searhc provides the `fit_surrogate(dataframe_of_results)` method for this use case. 

To simulate this we create a second evaluator `evaluator_2` and start a fresh AMBS search with strong explotation `kappa=0.001`.

In [17]:
evaluator_2 = get_evaluator(run)

search_from_checkpoint = AMBS(problem, evaluator_2, kappa=0.001)

# Initialize surrogate model of Bayesian optization (in AMBS)
# With results of previous search
search_from_checkpoint.fit_surrogate(results)

Created new evaluator with 1 worker and config: {'num_cpus': 1, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.LoggerCallback object at 0x7f87479c8820>]}


In [18]:
results_from_checkpoint = search_from_checkpoint.search(max_evals=10)

[00001] -- best objective: 0.7868852615356445 -- received objective: 0.7868852615356445
[00002] -- best objective: 0.7868852615356445 -- received objective: 0.7868852615356445
[00003] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00004] -- best objective: 0.8196721076965332 -- received objective: 0.8032786846160889
[00005] -- best objective: 0.8196721076965332 -- received objective: 0.7868852615356445
[00006] -- best objective: 0.8196721076965332 -- received objective: 0.8032786846160889
[00007] -- best objective: 0.8196721076965332 -- received objective: 0.8032786846160889
[00008] -- best objective: 0.8524590134620667 -- received objective: 0.8524590134620667
[00009] -- best objective: 0.8524590134620667 -- received objective: 0.7868852615356445
[00010] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889


In [19]:
results_from_checkpoint

,activation,batch_size,dropout_rate,learning_rate,num_epochs,units,id,objective,elapsed_sec,duration
0,tanh,8,0.295913,0.001246,21,25,1,0.786885,7.634545,7.389596
1,softplus,51,0.543369,0.000456,35,59,2,0.786885,10.548635,2.736648
2,softplus,9,0.336213,0.000995,31,97,3,0.819672,15.254908,4.530637
3,gelu,10,0.502697,0.000744,54,14,4,0.803279,19.885343,4.451506
4,linear,36,0.001525,0.003873,19,20,5,0.786885,22.680793,2.609599
5,hard_sigmoid,16,0.015906,0.002015,80,94,6,0.803279,27.812362,4.844054
6,hard_sigmoid,11,0.147917,0.001193,47,99,7,0.803279,32.108787,4.118992
7,selu,19,0.378121,0.000221,20,91,8,0.852459,34.828211,2.540145
8,softplus,14,0.184656,0.007196,19,84,9,0.786885,37.828185,2.824502
9,selu,10,0.453426,0.000101,24,84,10,0.803279,40.995007,2.992048


In [20]:
i_max = results_from_checkpoint.objective.argmax()
best_config = results_from_checkpoint.iloc[i_max][:-3].to_dict()

print(f"The default configuration has an accuracy of {objective_default:.3f}. " 
      f"The best configuration found by DeepHyper has an accuracy {results_from_checkpoint['objective'].iloc[i_max]:.3f}, " 
      f"trained in {results_from_checkpoint['duration'].iloc[i_max]:.2f} secondes and "
      f"finished after {results_from_checkpoint['elapsed_sec'].iloc[i_max]:.2f} secondes of search.")



best_config

The default configuration has an accuracy of 0.820. The best configuration found by DeepHyper has an accuracy 0.852, trained in 2.54 secondes and finished after 34.83 secondes of search.


{'activation': 'selu',
 'batch_size': 19,
 'dropout_rate': 0.3781208375459394,
 'learning_rate': 0.0002212993363258,
 'num_epochs': 20,
 'units': 91,
 'id': 8}

## Add conditionnal hyperparameters

Now we want to add the possibility to search for a second fully-connected layer. We simply add two new lines:

```python
if config.get("dense_2", False):
    x = tf.keras.layers.Dense(config["dense_2:units"], activation=config["dense_2:activation"])(x)
```

In [21]:
def run_with_condition(config: dict):
    tf.autograph.set_verbosity(0)
    
    train_dataframe, val_dataframe = load_data()

    train_ds = dataframe_to_dataset(train_dataframe)
    val_ds = dataframe_to_dataset(val_dataframe)

    train_ds = train_ds.batch(config["batch_size"])
    val_ds = val_ds.batch(config["batch_size"])

    # Categorical features encoded as integers
    sex = tf.keras.Input(shape=(1,), name="sex", dtype="int64")
    cp = tf.keras.Input(shape=(1,), name="cp", dtype="int64")
    fbs = tf.keras.Input(shape=(1,), name="fbs", dtype="int64")
    restecg = tf.keras.Input(shape=(1,), name="restecg", dtype="int64")
    exang = tf.keras.Input(shape=(1,), name="exang", dtype="int64")
    ca = tf.keras.Input(shape=(1,), name="ca", dtype="int64")

    # Categorical feature encoded as string
    thal = tf.keras.Input(shape=(1,), name="thal", dtype="string")

    # Numerical features
    age = tf.keras.Input(shape=(1,), name="age")
    trestbps = tf.keras.Input(shape=(1,), name="trestbps")
    chol = tf.keras.Input(shape=(1,), name="chol")
    thalach = tf.keras.Input(shape=(1,), name="thalach")
    oldpeak = tf.keras.Input(shape=(1,), name="oldpeak")
    slope = tf.keras.Input(shape=(1,), name="slope")

    all_inputs = [
        sex,
        cp,
        fbs,
        restecg,
        exang,
        ca,
        thal,
        age,
        trestbps,
        chol,
        thalach,
        oldpeak,
        slope,
    ]

    # Integer categorical features
    sex_encoded = encode_categorical_feature(sex, "sex", train_ds, False)
    cp_encoded = encode_categorical_feature(cp, "cp", train_ds, False)
    fbs_encoded = encode_categorical_feature(fbs, "fbs", train_ds, False)
    restecg_encoded = encode_categorical_feature(restecg, "restecg", train_ds, False)
    exang_encoded = encode_categorical_feature(exang, "exang", train_ds, False)
    ca_encoded = encode_categorical_feature(ca, "ca", train_ds, False)

    # String categorical features
    thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)

    # Numerical features
    age_encoded = encode_numerical_feature(age, "age", train_ds)
    trestbps_encoded = encode_numerical_feature(trestbps, "trestbps", train_ds)
    chol_encoded = encode_numerical_feature(chol, "chol", train_ds)
    thalach_encoded = encode_numerical_feature(thalach, "thalach", train_ds)
    oldpeak_encoded = encode_numerical_feature(oldpeak, "oldpeak", train_ds)
    slope_encoded = encode_numerical_feature(slope, "slope", train_ds)

    all_features = tf.keras.layers.concatenate(
        [
            sex_encoded,
            cp_encoded,
            fbs_encoded,
            restecg_encoded,
            exang_encoded,
            slope_encoded,
            ca_encoded,
            thal_encoded,
            age_encoded,
            trestbps_encoded,
            chol_encoded,
            thalach_encoded,
            oldpeak_encoded,
        ]
    )
    x = tf.keras.layers.Dense(config["units"], activation=config["activation"])(
        all_features
    )
    if config.get("dense_2", False):
        x = tf.keras.layers.Dense(config["dense_2:units"], activation=config["dense_2:activation"])(x)
    x = tf.keras.layers.Dropout(config["dropout_rate"])(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(all_inputs, output)

    optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])
    model.compile(optimizer, "binary_crossentropy", metrics=["accuracy"])

    history = model.fit(
        train_ds, epochs=config["num_epochs"], validation_data=val_ds, verbose=0
    )

    return history.history["val_accuracy"][-1]

To defined conditionnal hyperparameters we use [ConfigSpace](https://automl.github.io/ConfigSpace/master/index.html). We define `dense_2:units` and `dense_2:activation` as active hyperparameters only when `dense_2 == True`. The `cs.EqualsCondition` help us do that. Then we call

```python
problem_with_condition.add_condition(condition)
```

to register each new condition to the `HpProblem`.

In [22]:
import ConfigSpace as cs

# Define the same hyperparameters as before
problem_with_condition = HpProblem()
problem_with_condition.add_hyperparameter((8, 128), "units")
problem_with_condition.add_hyperparameter(ACTIVATIONS, "activation")
problem_with_condition.add_hyperparameter((0.0, 0.6), "dropout_rate")
problem_with_condition.add_hyperparameter((10, 100), "num_epochs")
problem_with_condition.add_hyperparameter((8, 256, "log-uniform"), "batch_size")
problem_with_condition.add_hyperparameter((1e-5, 1e-2, "log-uniform"), "learning_rate")

# Add a new hyperparameter "dense_2 (bool)" to decide if a second fully-connected layer should be created
hp_dense_2 = problem_with_condition.add_hyperparameter([True, False], "dense_2")
hp_dense_2_units = problem_with_condition.add_hyperparameter((8, 128), "dense_2:units")
hp_dense_2_activation = problem_with_condition.add_hyperparameter(ACTIVATIONS, "dense_2:activation")

problem_with_condition.add_condition(cs.EqualsCondition(hp_dense_2_units, hp_dense_2, True))
problem_with_condition.add_condition(cs.EqualsCondition(hp_dense_2_activation, hp_dense_2, True))


problem_with_condition

Configuration space object:
  Hyperparameters:
    activation, Type: Categorical, Choices: {elu, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softplus, softsign, swish, tanh}, Default: elu
    batch_size, Type: UniformInteger, Range: [8, 256], Default: 45, on log-scale
    dense_2, Type: Categorical, Choices: {True, False}, Default: True
    dense_2:activation, Type: Categorical, Choices: {elu, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softplus, softsign, swish, tanh}, Default: elu
    dense_2:units, Type: UniformInteger, Range: [8, 128], Default: 68
    dropout_rate, Type: UniformFloat, Range: [0.0, 0.6], Default: 0.3
    learning_rate, Type: UniformFloat, Range: [1e-05, 0.01], Default: 0.0003162278, on log-scale
    num_epochs, Type: UniformInteger, Range: [10, 100], Default: 55
    units, Type: UniformInteger, Range: [8, 128], Default: 68
  Conditions:
    dense_2:activation | dense_2 == True
    dense_2:units | dense_2 == True

We create a new evaluator `evaluator_3` and start a fresh AMBS search with this new problem `problem_with_condition`.

In [23]:
evaluator_3 = get_evaluator(run_with_condition)

search_with_condition = AMBS(problem_with_condition, evaluator_3)

Created new evaluator with 1 worker and config: {'num_cpus': 1, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.LoggerCallback object at 0x7f8747a25910>]}


In [24]:
results_with_condition = search_with_condition.search(max_evals=20)

[00001] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00002] -- best objective: 0.8196721076965332 -- received objective: 0.6557376980781555
[00003] -- best objective: 0.8196721076965332 -- received objective: 0.7377049326896667
[00004] -- best objective: 0.8196721076965332 -- received objective: 0.8196721076965332
[00005] -- best objective: 0.8196721076965332 -- received objective: 0.8032786846160889
[00006] -- best objective: 0.8524590134620667 -- received objective: 0.8524590134620667
[00007] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889
[00008] -- best objective: 0.8524590134620667 -- received objective: 0.688524603843689
[00009] -- best objective: 0.8524590134620667 -- received objective: 0.688524603843689
[00010] -- best objective: 0.8524590134620667 -- received objective: 0.6229507923126221
[00011] -- best objective: 0.8524590134620667 -- received objective: 0.8032786846160889
[00012] -- best objective: 0.85245

In [25]:
results_with_condition

,activation,batch_size,dense_2,dropout_rate,learning_rate,num_epochs,units,dense_2:activation,dense_2:units,id,objective,elapsed_sec,duration
0,tanh,27,False,0.256561,0.000117,79,15,NaN,NaN,1,0.819672,7.361361,7.171424
1,softsign,188,False,0.494313,0.000016,87,99,NaN,NaN,2,0.655738,11.200777,3.619191
2,hard_sigmoid,152,False,0.269453,0.000363,49,45,NaN,NaN,3,0.737705,14.658631,3.234346
3,tanh,34,True,0.271216,0.000011,97,77,tanh,26.0,4,0.819672,19.757613,4.868647
4,softsign,10,True,0.273368,0.000174,55,63,sigmoid,87.0,5,0.803279,24.842931,4.855399
5,tanh,8,False,0.563932,0.000013,97,42,NaN,NaN,6,0.852459,32.050291,6.985657
6,elu,8,True,0.390134,0.000014,42,89,elu,11.0,7,0.803279,37.850756,5.557407
7,relu,68,False,0.574432,0.000014,81,63,NaN,NaN,8,0.688525,42.979496,4.781515
8,softsign,15,False,0.463158,0.000011,87,11,NaN,NaN,9,0.688525,49.762694,6.417276
9,selu,224,True,0.127696,0.000023,60,82,selu,103.0,10,0.622951,53.507548,3.486371


Finally, let us print out the best configuration found from this conditionned search space.

In [26]:
i_max = results_with_condition.objective.argmax()
best_config = results_with_condition.iloc[i_max][:-3].to_dict()

print(f"The default configuration has an accuracy of {objective_default:.3f}. " 
      f"The best configuration found by DeepHyper has an accuracy {results_with_condition['objective'].iloc[i_max]:.3f}, " 
      f"trained in {results_with_condition['duration'].iloc[i_max]:.2f} secondes and "
      f"finished after {results_with_condition['elapsed_sec'].iloc[i_max]:.2f} secondes of search.")



best_config

The default configuration has an accuracy of 0.820. The best configuration found by DeepHyper has an accuracy 0.852, trained in 6.99 secondes and finished after 32.05 secondes of search.


{'activation': 'tanh',
 'batch_size': 8,
 'dense_2': False,
 'dropout_rate': 0.5639315153285624,
 'learning_rate': 1.3286479422843164e-05,
 'num_epochs': 97,
 'units': 42,
 'dense_2:activation': nan,
 'dense_2:units': nan,
 'id': 6}